In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D , BatchNormalization , Dropout , Dense
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint , LearningRateScheduler

In [2]:
HOME_TRAIN = False

In [3]:
BATCH_SIZE = 32
DROP_OUT_RATE = 0.2

In [4]:
dataset_info = pd.read_csv("meta_data_face_coor_K-Face.csv")
dataset_info

,file_path_x,left,right,top,bottom,연령대,성별,new_gender
0,C:/Users/Moon/Desktop/Age_Gender_Prediction/Da...,289,491,155,402,30대,남,1
1,C:/Users/Moon/Desktop/Age_Gender_Prediction/Da...,377,581,175,392,30대,남,1
2,C:/Users/Moon/Desktop/Age_Gender_Prediction/Da...,313,502,181,401,30대,남,1
3,C:/Users/Moon/Desktop/Age_Gender_Prediction/Da...,394,602,142,407,30대,남,1
4,C:/Users/Moon/Desktop/Age_Gender_Prediction/Da...,373,589,142,403,30대,남,1
...,...,...,...,...,...,...,...,...
365667,C:/Users/Moon/Desktop/Age_Gender_Prediction/Da...,323,635,101,470,20대,여,0
365668,C:/Users/Moon/Desktop/Age_Gender_Prediction/Da...,305,600,101,466,20대,여,0
365669,C:/Users/Moon/Desktop/Age_Gender_Prediction/Da...,282,577,101,467,20대,여,0
365670,C:/Users/Moon/Desktop/Age_Gender_Prediction/Da...,260,557,100,472,20대,여,0


In [20]:
data_file_path = dataset_info[['file_path_x' , 'left' , 'right' , 'top' , 'bottom']]
age = dataset_info['연령대'].tolist()

In [21]:
dataset_info['연령대'].value_counts()

40대    131725
30대    127951
50대     93736
20대     12260
Name: 연령대, dtype: int64

In [25]:
le_age = LabelEncoder()
le_age.fit(age)
print( le_age.classes_ )

le_age = le_age.transform(age)
age = tf.keras.utils.to_categorical(le_age, num_classes=4)

['20대' '30대' '40대' '50대']


In [26]:
age

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [28]:
file_path_train, file_path_val, y_train, y_val = train_test_split(data_file_path, age, 
                                                                  test_size=0.25, 
                                                                  random_state=777, 
                                                                  stratify = age)

In [29]:
print( len(file_path_train) , len(y_train) , len(file_path_val) , len(y_val) )

274254 274254 91418 91418


In [30]:
train_left = file_path_train['left'].tolist()
train_right = file_path_train['right'].tolist()
train_top = file_path_train['top'].tolist()
train_bottom = file_path_train['bottom'].tolist()
file_path_train = file_path_train['file_path_x'].tolist()

In [31]:
val_left = file_path_val['left'].tolist()
val_right = file_path_val['right'].tolist()
val_top = file_path_val['top'].tolist()
val_bottom = file_path_val['bottom'].tolist()
file_path_val = file_path_val['file_path_x'].tolist()

In [32]:
def load_image( image_path , left , right , top , bottom , label ):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)   
    img = tf.image.crop_to_bounding_box( img , top , left, bottom - top , right - left )
    
    """
    output_image = tf.image.encode_png(img)
    file_name = tf.constant('./Ouput_image.png')
    file = tf.io.write_file(file_name, output_image)    
    """
    
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.resnet50.preprocess_input(img)    
    
    return img , label

In [33]:
train_dataset = tf.data.Dataset.from_tensor_slices( (file_path_train , 
                                                     train_left , 
                                                     train_right , 
                                                     train_top , 
                                                     train_bottom , 
                                                     y_train) )

val_dataset = tf.data.Dataset.from_tensor_slices( (file_path_val , 
                                                   val_left , 
                                                   val_right , 
                                                   val_top , 
                                                   val_bottom ,
                                                   y_val) )

In [34]:
train_dataset = train_dataset.shuffle(buffer_size=len(file_path_train))\
                .map( load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                .batch(BATCH_SIZE)\
                .prefetch(tf.data.experimental.AUTOTUNE)     #


val_dataset = val_dataset.shuffle(buffer_size=len(file_path_val))\
                .map( load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                .batch(BATCH_SIZE)\
                .prefetch(tf.data.experimental.AUTOTUNE)    #

In [35]:
ResNet50 = tf.keras.applications.resnet.ResNet50(
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

In [36]:
model= Sequential()

model.add( ResNet50 )

model.add( GlobalAveragePooling2D() ) 
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(128, activation='relu') )
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 

model.add( Dense(4, activation='softmax') )

In [37]:
initial_learning_rate = 0.01

def lr_exp_decay(epoch, lr):
    k = 0.1
    return initial_learning_rate * np.math.exp(-k*epoch)

lr_scheduler = LearningRateScheduler(lr_exp_decay, verbose=1)

In [38]:
log_dir = os.path.join('Logs')
CHECKPOINT_PATH = os.path.join('CheckPoints_K-Face_AgeGroup')
tb_callback = TensorBoard(log_dir=log_dir)

cp = ModelCheckpoint(filepath=CHECKPOINT_PATH, 
                     monitor='val_accuracy',                     
                     save_best_only = True,
                     verbose = 1)

In [41]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [42]:
hist = model.fit(train_dataset,
                 validation_data=val_dataset,
                 callbacks=[lr_scheduler , cp , tb_callback],
                 epochs = 20,
                 verbose = 1 
)

Epoch 1/20

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
8571/8571 [==============================] - 1382s 161ms/step - loss: 1.1348 - accuracy: 0.4390 - val_loss: 1.1036 - val_accuracy: 0.5085

Epoch 00001: val_accuracy improved from -inf to 0.50849, saving model to CheckPoints_K-Face_AgeGroup


C:\Users\Moon\anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: CheckPoints_K-Face_AgeGroup\assets
Epoch 2/20

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009048374180359595.
8571/8571 [==============================] - 1375s 160ms/step - loss: 0.8107 - accuracy: 0.6454 - val_loss: 0.6045 - val_accuracy: 0.7498

Epoch 00002: val_accuracy improved from 0.50849 to 0.74983, saving model to CheckPoints_K-Face_AgeGroup


C:\Users\Moon\anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: CheckPoints_K-Face_AgeGroup\assets
Epoch 3/20

Epoch 00003: LearningRateScheduler reducing learning rate to 0.008187307530779819.
8571/8571 [==============================] - 1384s 161ms/step - loss: 0.3793 - accuracy: 0.8526 - val_loss: 0.4762 - val_accuracy: 0.8408

Epoch 00003: val_accuracy improved from 0.74983 to 0.84079, saving model to CheckPoints_K-Face_AgeGroup


...

INFO:tensorflow:Assets written to: CheckPoints_K-Face_AgeGroup\assets
Epoch 14/20

Epoch 00014: LearningRateScheduler reducing learning rate to 0.002725317930340126.
8571/8571 [==============================] - 1384s 161ms/step - loss: 0.0224 - accuracy: 0.9921 - val_loss: 1102.1735 - val_accuracy: 0.2515

Epoch 00014: val_accuracy did not improve from 0.98904
Epoch 15/20

Epoch 00015: LearningRateScheduler reducing learning rate to 0.0024659696394160645.
8571/8571 [==============================] - 1385s 161ms/step - loss: 0.0264 - accuracy: 0.9909 - val_loss: 0.0485 - val_accuracy: 0.9888

Epoch 00015: val_accuracy did not improve from 0.98904
Epoch 16/20

Epoch 00016: LearningRateScheduler reducing learning rate to 0.0022313016014842983.
8571/8571 [==============================] - 1387s 162ms/step - loss: 0.0163 - accuracy: 0.9940 - val_loss: 0.3981 - val_accuracy: 0.9817

Epoch 00016: val_accuracy did not improve from 0.98904
Epoch 17/20

Epoch 00017: LearningRateScheduler reducin

C:\Users\Moon\anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: CheckPoints_K-Face_AgeGroup\assets
Epoch 20/20

Epoch 00020: LearningRateScheduler reducing learning rate to 0.0014956861922263505.
8571/8571 [==============================] - 1386s 162ms/step - loss: 0.0159 - accuracy: 0.9944 - val_loss: 0.0288 - val_accuracy: 0.9926

Epoch 00020: val_accuracy improved from 0.99093 to 0.99259, saving model to CheckPoints_K-Face_AgeGroup


C:\Users\Moon\anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: CheckPoints_K-Face_AgeGroup\assets


# 사용한 Code

In [ ]:
def gender_map( gender ):
    if gender == '남':
        return 1
    elif gender == '여':
        return 0

In [ ]:
dataset_info['new_gender'] = dataset_info['성별'].map( gender_map )
dataset_info.head()

In [ ]:
if HOME_TRAIN == False:
    data_file_path = [c.replace('f:/', 'C:/Users/Moon/Desktop/Age_Gender_Prediction/Dataset/') for c in data_file_path]

In [ ]:
def get_ID_2(file_path):
    return file_path[19:]

In [ ]:
dataset_info['ID'] = dataset_info['file_path'].apply(get_ID_2)
dataset_info['ID']

In [ ]:
dataset_info.info()

In [ ]:
face_coor_info = pd.read_csv("coor_211129.csv")

In [ ]:
face_coor_info.head()

In [ ]:
face_coor_info.info()

In [ ]:
def get_ID(file_path):
    return file_path[68:]

In [ ]:
face_coor_info['file_path'][0]

In [ ]:
face_coor_info['ID'] = face_coor_info['file_path'].apply(get_ID)
face_coor_info['ID']

In [ ]:
meta_data_face_coor = pd.merge(face_coor_info , dataset_info , how='left' , on='ID')

In [ ]:
meta_data_face_coor.head()

In [ ]:
meta_data_face_coor.info()

In [ ]:
meta_data_face_coor = meta_data_face_coor.drop(['file_path_y'] , axis=1)

In [ ]:
meta_data_face_coor.head()

In [ ]:
meta_data_face_coor.isna().sum()

In [ ]:
meta_data_face_coor['연령대'].value_counts()

In [ ]:
meta_data_face_coor['new_gender'].value_counts()

In [ ]:
meta_data_face_coor.to_csv("meta_data_face_coor_K-Face.csv",index=False)